In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
KEY = os.getenv("OPEN_AI_KEY")
print(KEY)

sk-proj-Zi8CDRosgQ5N4P8qPqvoT3BlbkFJuuVI3RApo5raDzPJOR4I


In [5]:
import langchain
from langchain_community.chat_models import ChatOpenAI
from langchain.chat_models import ChatOpenAI

In [6]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

d:\ML\Generative_AI\Project1\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [8]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
}

In [9]:
TEMPLATE= """
Text:{text}
You are an expert M maker. Given the above text, it is your job to\
    create a quize of {number} multiple choice questions for {subject} students in {tone} tone.
    Make sure the questions are not repeated and check all the questions to be conforming the text as well.
    Make sure to forma your response like RESPONSE_JSON bellow and use it as a guid.\
    Ensure to make {number} MCQ
    ### RESPONSE_JSON
    {response_json}
"""

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [11]:
#llm chain use to connect verious component
quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

d:\ML\Generative_AI\Project1\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [12]:
TEMPLATE2= """
your are an expert english grammarin and writer. Given a Multiple choice quiz for {subject} students. \
    You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use maximum of 50 for complexity.
    It the quiz is not at per with the cognitive and analytic abilities of the students,\
    update the quiz questions which needs to be changed and change the tone such that it perfectly fits the sudent's ability.
    Quiz_MCQs:
    {quiz}
    check from an expert English writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables = ["subject","quiz"],
    template = TEMPLATE2
)

In [14]:
review_chain =LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [15]:
# made 2 chains now we will connect it via sequential chain
generate_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
                                output_variables=["quiz","review"],verbose=True)

In [16]:
file_path = "D:\ML\Generative_AI\Project1\Data.txt"
file_path

'D:\\ML\\Generative_AI\\Project1\\Data.txt'

In [17]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [18]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [19]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [20]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [21]:
# get_openai_callback - helps track all the tokens
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON),
        }
    )

NameError: name 'generate_evaluate_chain' is not defined

In [ ]:
quiz = response.get("quiz")

NameError: name 'response' is not defined

In [ ]:
json.loads(quiz)

NameError: name 'quiz' is not defined

In [22]:
quiz_table_data = []
for key,value in quiz.items():
    mcq=value["mcq"]
    options = " | ".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value['optons'].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"Correct":correct})

NameError: name 'quiz' is not defined

In [ ]:
quiz = pd.DataFrame(quiz_table_data)

In [23]:
quiz.to_csv("ml.csv")

NameError: name 'quiz' is not defined

In [25]:
from datetime import datetime

LOG_FILE = f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}"
print(LOG_FILE)

06_25_2024_18_32_59
